In [1]:
from datasets import Dataset, load_dataset, concatenate_datasets

dataset = load_dataset("lmsys/chatbot_arena_conversations")
dataset = dataset["train"]

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
set(dataset["model_a"])

{'RWKV-4-Raven-14B',
 'alpaca-13b',
 'chatglm-6b',
 'claude-instant-v1',
 'claude-v1',
 'dolly-v2-12b',
 'fastchat-t5-3b',
 'gpt-3.5-turbo',
 'gpt-4',
 'gpt4all-13b-snoozy',
 'guanaco-33b',
 'koala-13b',
 'llama-13b',
 'mpt-7b-chat',
 'oasst-pythia-12b',
 'palm-2',
 'stablelm-tuned-alpha-7b',
 'vicuna-13b',
 'vicuna-7b',
 'wizardlm-13b'}

In [8]:
set(dataset["language"])

{'Afrikaans',
 'Arabic',
 'Aymara',
 'Bulgarian',
 'Catalan',
 'Chinese',
 'Croatian',
 'Czech',
 'Danish',
 'Dutch',
 'English',
 'Esperanto',
 'Estonian',
 'Fijian',
 'Finnish',
 'French',
 'Galician',
 'German',
 'Greek',
 'Guarani',
 'Haitian Creole',
 'Hawaiian',
 'Hebrew',
 'Hmong',
 'Hungarian',
 'Indonesian',
 'Interlingua',
 'Italian',
 'Japanese',
 'Klingon',
 'Korean',
 'Latin',
 'Latvian',
 'Luxembourgish',
 'Macedonian',
 'Malay',
 'Maltese',
 'Norwegian',
 'Norwegian Nynorsk',
 'Persian',
 'Polish',
 'Portuguese',
 'Quechua',
 'Romanian',
 'Russian',
 'Sanskrit',
 'Scots',
 'Serbian',
 'Shona',
 'Slovak',
 'Slovenian',
 'Spanish',
 'Swedish',
 'Thai',
 'Tsonga',
 'Turkish',
 'Ukrainian',
 'Uzbek',
 'Vietnamese',
 'Waray',
 'Xhosa',
 'unknown'}

In [2]:
include_model = lambda x: x in ["gpt-4", "claude-v1", "gpt-3.5-turbo"]

dataset = dataset.filter(
    lambda x: (include_model(x["model_a"]) or include_model(x["model_b"]))
    # and x["language"] == "Russian"
)

In [3]:
dataset

Dataset({
    features: ['question_id', 'model_a', 'model_b', 'winner', 'judge', 'conversation_a', 'conversation_b', 'turn', 'anony', 'language', 'tstamp', 'openai_moderation', 'toxic_chat_tag'],
    num_rows: 11762
})

In [4]:
dataset[10]

{'question_id': 'c22fc80a3e9a4c31a7f1c5005ab0ce6f',
 'model_a': 'vicuna-13b',
 'model_b': 'gpt-3.5-turbo',
 'winner': 'model_b',
 'judge': 'arena_user_4055',
 'conversation_a': [{'content': "Explain string theory like I'm 5 years old",
   'role': 'user'},
  {'content': "String theory is a way that some scientists try to understand the universe. They think that everything in the universe, including you and me, is made up of tiny strings that vibrate at different frequencies. These vibrations create all the different types of particles, like atoms and photons, that make up everything around us.\n\nThe strings are too small to see with our eyes, but scientists use special tools, like telescopes and microscopes, to study them. They also use math to help them understand how the strings work and how they create different types of particles.\n\nString theory is still a theory, which means that scientists are still trying to prove it's true. But if it turns out to be true, it could help us und

In [9]:
new_dataset = []

for item in dataset:
    question_id = item["question_id"]
    model_a = item["model_a"]
    model_b = item["model_b"]
    if include_model(model_a):
        new_dataset.append(
            {
                "question_id": question_id,
                "model_a_b": "model_a",
                "conversation": item["conversation_a"],
                "model_name": model_a
            }
        )
    if include_model(model_b):
        new_dataset.append(
            {
                "question_id": question_id,
                "model_a_b": "model_b",
                "conversation": item["conversation_b"],
                "model_name": model_b
            }
        )

In [10]:
len(new_dataset)

12798

In [11]:
new_dataset = Dataset.from_list(new_dataset)

In [12]:
new_dataset[0]

{'question_id': 'daab6467e9534a87af17bab7a66c6add',
 'model_a_b': 'model_b',
 'conversation': [{'content': 'hablame en español', 'role': 'user'},
  {'content': 'Claro, ¿en qué puedo ayudarte?', 'role': 'assistant'},
  {'content': 'que es la matrix?', 'role': 'user'},
  {'content': 'En el contexto de la ciencia ficción, la "Matrix" es un término que se refiere a una realidad simulada o virtual en la que los seres humanos viven sin saber que su mundo es una ilusión generada por una inteligencia artificial. En la película "The Matrix", la humanidad ha sido esclavizada por máquinas y se les ha implantado una ilusión de la vida real para controlarlos. La trama se centra en la lucha de un grupo de rebeldes humanos contra las máquinas y en la búsqueda de la verdad sobre la realidad en la que viven.',
   'role': 'assistant'}],
 'model_name': 'gpt-3.5-turbo'}

In [13]:
new_dataset.push_to_hub("dim/lmsys_chatbot_arena_conversations_gpt4_gpt-3.5-turbo_claudy")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:03<00:00,  3.51s/it]
